# <b style='color:blue;'> Random Forest </b>
* Este Jupyter tem como função treinar um `Random Forest` com os melhores parametros determinados por um `RandomSearch`.

-> Dataset completo com voluntários femininos, sem outliers e com a coluna de semanas, mas ainda predizendo o label. Sem pca.

---

Bibliotecas/Módulos

In [1]:
import logging
import numpy as np
import pandas as pd
import pickle as pkl
import sys

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef, confusion_matrix, classification_report, f1_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split

Configurações dos logs

In [2]:
logger = logging.getLogger("SVM")
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('%(name)s - %(levelname)s - [+] ------- %(message)s -------') 
handler.setFormatter(formatter)
logger.handlers = [handler]

Cores

In [3]:
RED = "\033[1;31m"
BLUE = "\033[1;34m"
GREEN = "\033[1;32m"
PINK = "\033[1;45m"
HEADER = '\033[95m'
OKBLUE = '\033[94m'
OKCYAN = '\033[96m'
OKGREEN = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
ENDC = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
MAG = "\033[1;45m"

---

## Preparando os dados

In [4]:
df = pd.read_csv("dados_FEM_10_voluntarios_c_semanas.csv")

In [5]:
df.sample()

,0,1,2,3,4,5,6,7,8,9,...,1992,1993,1994,1995,1996,1997,1998,1999,label,semana
1482,-32,-9,-59,48,-30,0,-36,-1,-7,-21,...,-21,20,40,26,-62,15,8,7,1,2


Verificando tamanho do datasetSeparando os dados em dados de treino e teste

In [6]:
len(df)

38400

* Seleção da variável target e das features

In [7]:
y = df["label"]

In [8]:
X = df.drop(["label"], axis=1)

Separando os dados em dados de treino e teste

In [9]:
x_treino, x_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=7)

---

## Treinamento

### RandomSeach

Parametros do modelo escolhidos para otimização

In [10]:
values = {'n_estimators':[1, 100, 1000], 'min_samples_leaf': [1, 3], 'n_jobs':[1]}

In [25]:
clf = RandomForestClassifier()

In [12]:
def best_params(values: dict, clf: 'classificador', x_treino: np.array, x_teste: np.array, y_treino: np.array, y_teste: np.array) -> list:
    '''
        Instancia um classificador de busca e procura os melhores parâmetros para o modelo.
        
        Params
        ------
        :values: parametros a serem testados
        :clf: instancia do classificador desejado
        :x_treino: dados de treino
        :x_teste: dados de teste
        :y_treino: dados de treino -labels
        :y_teste: dados de treino - labels
        
        Return
        ------
        :best_params: lista com os melhores parametros
    '''
    
    logger.info('Instanciando RF')
    random_clf = RandomizedSearchCV(clf, param_distributions=values, n_iter=200)
    
    logger.info('Treinando RF')
    random_clf.fit(x_treino, y_treino)
    
    logger.info('Predict RF')
    y_random_clf_rl = random_clf.predict(x_teste)
    
    logger.info('Parametros RF')
    print(RED, random_clf.get_params())
    
    logger.info('Best Params RF')
    best_params = random_clf.best_params_
    print(BLUE, best_params)
    
    return best_params

In [13]:
%%time

try:
    logging.info("teste")
    best = best_params(values, clf, x_treino, x_teste, y_treino, y_teste)

except Exception as error:
    logger.warning('Aconteceu algum problema...')
    logger.critical(error)

else:
    logger.info('Ok')
    
finally:
    logger.info('Busca finalizada')

SVM - INFO - [+] ------- Instanciando RF -------
INFO:SVM:Instanciando RF
SVM - INFO - [+] ------- Treinando RF -------
INFO:SVM:Treinando RF
/home/saraselis/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 6 is smaller than n_iter=200. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
SVM - INFO - [+] ------- Predict RF -------
INFO:SVM:Predict RF
SVM - INFO - [+] ------- Parametros RF -------
INFO:SVM:Parametros RF
SVM - INFO - [+] ------- Best Params RF -------
INFO:SVM:Best Params RF
SVM - INFO - [+] ------- Ok -------
INFO:SVM:Ok
SVM - INFO - [+] ------- Busca finalizada -------
INFO:SVM:Busca finalizada


 {'cv': None, 'error_score': nan, 'estimator__bootstrap': True, 'estimator__ccp_alpha': 0.0, 'estimator__class_weight': None, 'estimator__criterion': 'gini', 'estimator__max_depth': None, 'estimator__max_features': 'auto', 'estimator__max_leaf_nodes': None, 'estimator__max_samples': None, 'estimator__min_impurity_decrease': 0.0, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__n_estimators': 100, 'estimator__n_jobs': None, 'estimator__oob_score': False, 'estimator__random_state': None, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': RandomForestClassifier(), 'n_iter': 200, 'n_jobs': None, 'param_distributions': {'n_estimators': [1, 100, 1000], 'min_samples_leaf': [1, 3], 'n_jobs': [1]}, 'pre_dispatch': '2*n_jobs', 'random_state': None, 'refit': True, 'return_train_score': False, 'scoring': None, 'verbose': 0}
 {'n_jobs': 1, 'n_estimators': 1000, 'min_samples_leaf': 1}
CPU times: user 2h 46

Melhores parâmetros:

{'n_jobs': 1, 'n_estimators': 1000, 'min_samples_leaf': 1}

In [ ]:
best

----

### Random Forest

In [10]:
def classificador(x_treino: np.array, y_treino: np.array, x_teste: np.array, y_teste: np.array):
    '''
        Treina um classificador com os melhores parametros possíveis.
        
        Params
        ------
        :x_treino: dados de treino
        :x_teste: dados de teste
        :y_treino: dados de treino -labels
        :y_teste: dados de treino - labels
    
        Return
        ------
        :RF: Modelo Random Forest treinado
        :resultados (matrix): métricas
    '''
    
    logger.info('Instanciando Random Forest')
    RF = RandomForestClassifier(n_jobs= 1, n_estimators= 1000, min_samples_leaf= 1)
    
    logger.info('Treinando Random Forest')
    RF.fit(x_treino, y_treino)
    
    logger.info('Predict')
    p = RF.predict_proba(x_teste)[:, 1]
    categorias = RF.predict(x_teste)
    
    logger.info('Métricas: \n \n')
    
    logger.info('Roc')
    roc = roc_auc_score(y_teste, p)
    print(OKBLUE, "Roc \n" + str(roc) + "\n")
    
    logger.info('Acurácia')
    acc = accuracy_score(y_teste, categorias)
    print(OKCYAN, "Acc \n" + str(acc) + "\n")
    
    logger.info("F1")
    f1 = f1_score(y_teste, categorias, average="weighted")
    print(GREEN, "F1 \n" + str(f1) + "\n")
    
    logger.info("Mcc")
    mcc = matthews_corrcoef(y_teste, categorias)
    print(WARNING, "Mcc \n" + str(mcc) + "\n")
    
    logger.info('Class Report')
    resultados = classification_report(y_teste, categorias)
    print(ENDC, "Class Report \n" + resultados + "\n")
    
    return RF, resultados

In [11]:
%%time

try:
    RF, resultados = classificador(
                        x_treino,
                        y_treino,
                        x_teste,
                        y_teste)

except Exception as error:
    logger.warning('Aconteceu algum problema...')
    logger.critical(error)

else:
    logger.info('Ok')
    
finally:
    logger.info('Treino finalizado')

SVM - INFO - [+] ------- Instanciando Random Forest -------
SVM - INFO - [+] ------- Treinando Random Forest -------
SVM - INFO - [+] ------- Predict -------
SVM - INFO - [+] ------- Métricas: 
 
 -------
SVM - INFO - [+] ------- Roc -------
SVM - INFO - [+] ------- Acurácia -------
SVM - INFO - [+] ------- F1 -------
SVM - INFO - [+] ------- Mcc -------
SVM - INFO - [+] ------- Class Report -------
SVM - INFO - [+] ------- Ok -------
SVM - INFO - [+] ------- Treino finalizado -------


 Roc 
0.9104168436974651

 Acc 
0.7924479166666667

 F1 
0.7924303491896499

 Mcc 
0.5850305193604062

 Class Report 
              precision    recall  f1-score   support

           0       0.79      0.80      0.79      3833
           1       0.80      0.78      0.79      3847

    accuracy                           0.79      7680
   macro avg       0.79      0.79      0.79      7680
weighted avg       0.79      0.79      0.79      7680


CPU times: user 19min 26s, sys: 0 ns, total: 19min 26s
Wall time: 19min 26s


----

#### Exportando modelo

In [14]:
pkl.dump(RF, open('modelos/rf_dados_FEM_10_voluntarios_c_semanas_c_coluna_semana.pkl', 'wb'))